![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [107]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below

In [108]:
df = pd.read_csv('rental_info.csv')

In [109]:
df['rental_date'] = pd.to_datetime(df['rental_date'])
df['return_date'] = pd.to_datetime(df['return_date'])


In [110]:
df['rental_lenght_days'] = (df['return_date'] - df['rental_date']).dt.days

In [111]:
df['deleted_scenes'] = df['special_features'].apply(lambda x: 1 if 'Deleted Scenes' in x else 0)
df['behind_the_scenes'] = df['special_features'].apply(lambda x: 1 if 'Behind the Scenes' in x else 0)


In [112]:
X = df.select_dtypes(include=['number']).drop(columns = 'rental_lenght_days')

In [113]:
y = df.rental_lenght_days

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 9)

In [115]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [116]:
scaler = MinMaxScaler()
scaler.fit_transform(X_train)


array([[0.18181818, 1.        , 0.5       , ..., 0.33277592, 0.        ,
        1.        ],
       [0.36363636, 0.66666667, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.54545455, 0.5       , 1.        , ..., 1.        , 0.        ,
        1.        ],
       ...,
       [0.45454545, 0.16666667, 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.18181818, 0.33333333, 0.5       , ..., 0.33277592, 0.        ,
        0.        ],
       [0.45454545, 0.16666667, 0.5       , ..., 0.33277592, 1.        ,
        1.        ]])

In [121]:
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import RandomizedSearchCV

In [128]:
param_dist = {'fit_intercept':[True, False]}

lr = LinearRegression()

rand_search = RandomizedSearchCV(lr, param_distributions=param_dist, cv=3, random_state=42, n_iter=6)

rand_search.fit(X_train, y_train)
hyper_params = rand_search.best_params_

lr = LinearRegression(fit_intercept=hyper_params["fit_intercept"])
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

mse_random = mean_squared_error(y_test, lr_pred)

In [129]:
best_mse

2.225667528098759